In [3]:
# --- Setup: always run this first ---
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Helper function: run circuit & plot histogram
def run_and_plot(qc, shots=1024):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    result = sim.run(tqc, shots=shots).result()
    counts = result.get_counts()
    plot_histogram(counts)
    plt.show()
    return counts


In [4]:
# Grover’s Algorithm: Search for a marked state

n = 3                      # number of qubits
marked = '101'             # the state we want to find

# Oracle: marks the state |marked>
def grover_oracle(marked):
    n = len(marked)
    qc = QuantumCircuit(n, name='oracle')
    
    # Flip qubits where marked bit is '0'
    for i, bit in enumerate(reversed(marked)):
        if bit == '0':
            qc.x(i)
    
    # Multi-controlled Z (via H, MCX, H)
    qc.h(n-1)
    qc.mcx(list(range(n-1)), n-1, mode='noancilla')
    qc.h(n-1)
    
    # Undo the X gates
    for i, bit in enumerate(reversed(marked)):
        if bit == '0':
            qc.x(i)
    
    return qc

# Diffuser (inversion about mean)
def diffuser(n):
    qc = QuantumCircuit(n, name='diffuser')
    qc.h(range(n))
    qc.x(range(n))
    qc.h(n-1)
    qc.mcx(list(range(n-1)), n-1, mode='noancilla')
    qc.h(n-1)
    qc.x(range(n))
    qc.h(range(n))
    return qc

# Build Grover circuit
qc = QuantumCircuit(n, n)

# Step 1: put qubits into superposition
qc.h(range(n))

# Step 2: apply oracle
qc.append(grover_oracle(marked).to_gate(), range(n))

# Step 3: apply diffuser
qc.append(diffuser(n).to_gate(), range(n))

# Step 4: measure
qc.measure(range(n), range(n))

print(qc.draw(output="text"))
counts_g = run_and_plot(qc, shots=2048)
print("Most frequent result:", max(counts_g, key=counts_g.get))


C:\Users\liinvwp\AppData\Local\Temp\ipykernel_13184\1120022281.py:18: DeprecationWarning: ``qiskit.circuit.quantumcircuit.QuantumCircuit.mcx()``'s argument ``mode`` is deprecated as of Qiskit 2.1. It will be removed no earlier than 3 months after the release date. Instead, add a generic MCXGate to the circuit and specify the synthesis method via the ``hls_config`` in the transpilation. Alternatively, specific decompositions are available at https://qisk.it/mcx.
  qc.mcx(list(range(n-1)), n-1, mode='noancilla')
C:\Users\liinvwp\AppData\Local\Temp\ipykernel_13184\1120022281.py:34: DeprecationWarning: ``qiskit.circuit.quantumcircuit.QuantumCircuit.mcx()``'s argument ``mode`` is deprecated as of Qiskit 2.1. It will be removed no earlier than 3 months after the release date. Instead, add a generic MCXGate to the circuit and specify the synthesis method via the ``hls_config`` in the transpilation. Alternatively, specific decompositions are available at https://qisk.it/mcx.
  qc.mcx(list(rang

     ┌───┐┌─────────┐┌───────────┐┌─┐      
q_0: ┤ H ├┤0        ├┤0          ├┤M├──────
     ├───┤│         ││           │└╥┘┌─┐   
q_1: ┤ H ├┤1 oracle ├┤1 diffuser ├─╫─┤M├───
     ├───┤│         ││           │ ║ └╥┘┌─┐
q_2: ┤ H ├┤2        ├┤2          ├─╫──╫─┤M├
     └───┘└─────────┘└───────────┘ ║  ║ └╥┘
c: 3/══════════════════════════════╩══╩══╩═
                                   0  1  2 
Most frequent result: 101
